# Barcodes

Show that barcodes don't matter and that we can pick out when they do.

**Note: first run of this cell takes a while**

In [1]:
%load_ext autoreload
%autoreload 2  
%reload_ext autoreload
import sys
sys.path.append('../../src')
from util import *
from config import *
from study_gen import study
import generate_dataset
import ipynbname
import plots

Reading study from df.feather...
Done reading study from df.feather.


In [2]:
study.df['sample'].unique()

array(['65degrees_1_S20_L001', 'Lauren_603_1min_S6_L001',
       '5degrees_2_S9_L001', 'Lauren_601_05_S1_L001', '1min_1_S14_L001',
       '2percent_1_S28_L001', '05_1_S24_reads', '36DMS_S2',
       '5percent_2_S31_L001', 'lauren474_S5', '19_DMS_S9_L001',
       '2_1_s28new_reads', '37degrees_01percent_2_S17_L001',
       'Lauren_601_2_S2_L001', '25degrees_2_S13_L001', '19_RNA_S10_L001',
       '37degrees_1percent_2_S15_L001', '1_1_S26new_reads',
       '10_1_s32new_reads', '37degrees_1percent_1_S14_L001',
       '25degrees_1_S12_L001', '5percent_1_S30_L001',
       '37degrees_01percent_1_S16_L001', 'IDX-41_S10_L001',
       '1min_2_S15_L001', '2percent_2_S29_L001', '01_02_S23_reads',
       'lauren470_S1', '65degrees_2_S21_L001', '5_1_S30new_reads',
       '5degrees_1_S8_L001', '05percent_2_S25_L001', 'lauren473_S4',
       'Lauren_603_3min_S7_L001', '01percent_2_S23_L001',
       '19_plasmid_S12_L001', '10min_1_S20_L001', '1percent_2_S27_L001',
       '5min_2_S19_L001', '10percent_1_S

### Barcodes comparison scatter plot
- x-axis = barcode1 
- y-axes = barcode2
- fit: 
    - x=y (line of identity)
    - best-fit line
    - Pearson R
- if more than 2 barcode reps then heatmap pearson R everything vs. everything 

In [3]:
## Change sample ##
sample = '18_DMS_S7_L001'   
##############################    

%reload_ext autoreload
plot = plots.barcode_comparison_scatter_plot(study, sample)

save_plotly_fig(
    ipynbname.path(), 
    '[A] Barcodes comparison scatter plot/{}'.format(sample),
    plot['fig']
    )

plot['fig'].show()
plot['data']

AssertionError: Attribute reference not found in dataframe

## Barcode replicates simulation
### Experimental: what distribution to expect?

In [ ]:
plot = plots.barcode_comparison_scatter_plot(study, sample)


AssertionError: No barcode replicates found for sample 18_DMS_S7_L001

### Combined barcode reps in sample
- x-axis = barcode1/barcode2 names 
- y-axis = pearson R 
- ranked by R
- Or is there a better way to do this–% percent total??


In [ ]:

plot = plots.combined_barcode_replicates_in_sample(study)
save_plotly_fig(ipynbname.path(), '[B] Combined barcode reps in sample', plot['fig'])
plot['fig'].show()


/Users/ymdt/src/highthroughputcellularbiology/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

/Users/ymdt/src/highthroughputcellularbiology/venv/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars

/Users/ymdt/src/highthroughputcellularbiology/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

/Users/ymdt/src/highthroughputcellularbiology/venv/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars

/Users/ymdt/src/highthroughputcellularbiology/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

/Users/ymdt/src/highthroughputcellularbiology/venv/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars

/Users/ymdt/src/highthroughputcellula

In [ ]:
import plotly.graph_objects as go
from scipy import stats

n=1000
n1 = 510
n2 = 1000
p1 = 0.012
p2 = 0.01
P1 = np.random.binomial(n1, p1, n)/n1
P2 = np.random.binomial(n2, p2, n)/n2

# Calculate the Fisher's exact test
def fisher_exact_test(n1, p1, n2, p2):
    # Calculate the odds ratio
    oddsratio, pvalue = stats.fisher_exact([[n1*p1, n1*(1-p1)], [n2*p2, n2*(1-p2)]])
    return oddsratio, pvalue

# Combine the p-values using the Fisher's method
def compute_combined_pvalue_from_mut_rates(n1, n2, mr1, mr2):
    pval = []
    for p1, p2 in zip(mr1, mr2):
        oddsratio, pvalue = fisher_exact_test(n1, p1, n2, p2)
        pval.append(pvalue)
    all_pvalues = -2 * np.sum(np.log(pval))
    return 1 - stats.chi2.cdf(all_pvalues, 2 * len(pval))

out = pd.DataFrame(columns=['pvalue','n1','n2'])
for _ in range(10):
    # Generate random mutation rates
    P1 = np.random.binomial(n1, p1, n)/n1
    P2 = np.random.binomial(n2, p2, n)/n2
    
    # Calculate the combined p-value
    pvalue = compute_combined_pvalue_from_mut_rates(n1, n2, P1, P2)

    # Store the results
    out.append(pvalue)


# Plot the results
fig = go.Figure()
fig.add_trace(go.Histogram(x=out, histnorm='probability density'))
fig.update_layout(
    title='Combined p-value from mutation rates',
    xaxis_title='p-value',
    yaxis_title='Probability density',
    bargap=0.2,
    bargroupgap=0.1
)


